## 1. Introdução e Carregamento

Começamos importando as bibliotecas necessárias e carrregando os dataset's brutos (carga, tempo e clima). Em seguida usamos o info() para obter informações como dtypes de cada label e entender os nulls presentes.

In [60]:
!wget https://raw.githubusercontent.com/owsei-data/abtra-climate-impact/refs/heads/main/data/movimentacao-carga-2019-2025%20-%20Carga.csv
!wget https://raw.githubusercontent.com/owsei-data/abtra-climate-impact/refs/heads/main/data/movimentacao-tempos-2019-2025%20-%20Tempos.csv
!wget https://raw.githubusercontent.com/owsei-data/abtra-climate-impact/refs/heads/main/data/weather-bertioga-2019-2025.csv

--2025-11-21 16:48:23--  https://raw.githubusercontent.com/owsei-data/abtra-climate-impact/refs/heads/main/data/movimentacao-carga-2019-2025%20-%20Carga.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6443 (6.3K) [text/plain]
Saving to: ‘movimentacao-carga-2019-2025 - Carga.csv’

movimentacao-carga- 100%[===================>]   6.29K  --.-KB/s    in 0s      

2025-11-21 16:48:23 (67.1 MB/s) - ‘movimentacao-carga-2019-2025 - Carga.csv’ saved [6443/6443]

--2025-11-21 16:48:23--  https://raw.githubusercontent.com/owsei-data/abtra-climate-impact/refs/heads/main/data/movimentacao-tempos-2019-2025%20-%20Tempos.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubu

In [61]:
import pandas as pd

carga = pd.read_csv('movimentacao-carga-2019-2025 - Carga.csv')
tempo = pd.read_csv('movimentacao-tempos-2019-2025 - Tempos.csv')
clima = pd.read_csv('weather-bertioga-2019-2025.csv')

In [62]:
carga.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 164 entries, 0 to 163
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   mes            164 non-null    int64 
 1   ano            164 non-null    int64 
 2   sit.carga      164 non-null    object
 3   qtde.unidades  164 non-null    int64 
 4   qtde.teus      164 non-null    int64 
 5   peso.bruto     164 non-null    object
 6   peso.liquido   164 non-null    object
dtypes: int64(4), object(3)
memory usage: 9.1+ KB


In [63]:
tempo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 164 entries, 0 to 163
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   mes                         164 non-null    int64 
 1   ano                         164 non-null    int64 
 2   tipo.documento              164 non-null    int64 
 3   total.ctr                   164 non-null    int64 
 4   entrada.ate.doc.import      164 non-null    object
 5   doc.import.ate.desembaraco  164 non-null    object
 6   desembaraco.ate.saida       164 non-null    object
 7   entrada.ate.saida           164 non-null    object
 8   operador                    164 non-null    object
 9   transito                    164 non-null    object
 10  recinto                     164 non-null    object
 11  porto                       164 non-null    object
dtypes: int64(4), object(8)
memory usage: 15.5+ KB


In [64]:
clima.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82 entries, 0 to 81
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Ano            82 non-null     int64  
 1   Mes            82 non-null     int64  
 2   Chuva_Total    49 non-null     float64
 3   Chuva_Max_Dia  49 non-null     float64
 4   Vento_Max_Mes  29 non-null     float64
 5   Vento_Media    29 non-null     float64
 6   Pct_Chuva      82 non-null     float64
 7   Pct_Vento      82 non-null     float64
dtypes: float64(6), int64(2)
memory usage: 5.3 KB


## 2. Limpeza dos Nomes das Colunas

Notamos que algumas labels podem conter espaços em branco extrars no ínicio ou no fim dos nomes, o que poderia atrapalhar a seleção dos dados. Porr isso, fizemos uma limpeza rápida removendo esses espaços (strip) pra deixar tudo padronizado.

In [65]:
#removendo possiveis espaços extras nas labels
carga.columns = carga.columns.str.strip()
tempo.columns = tempo.columns.str.strip()
clima.columns = clima.columns.str.strip()

In [66]:
#dimensoes dataset
print(carga.shape)
print(tempo.shape)
print(clima.shape)

#o dataset da abtra tem duas linhas pra cada mes e ano

(164, 7)
(164, 12)
(82, 8)


## 3. Tratamento de Tipos Numéricos

Os dados numéricos da ABTRA foram lidos como texto (object) porque estavam no formato brasileiro, usando ponto para milhar e virgula para decimal. Criamos aqui a função coluna_numerica para limpar essa formatação e converter tdo para (float).

In [67]:
def coluna_numerica(label):
    if label.dtype == 'object':
        #remove o ponto de milhar e troca virgula por ponto
        label = label.str.replace('.', '', regex=False).str.replace(',', '.', regex=False)
        return pd.to_numeric(label, errors='coerce')
    return label

In [68]:
#usar coluna_numerica na carga pras medidas
coluna_numerica_carga = ['peso.bruto', 'peso.liquido']

for col in coluna_numerica_carga:
    carga[col] = coluna_numerica(carga[col])

carga.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 164 entries, 0 to 163
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   mes            164 non-null    int64  
 1   ano            164 non-null    int64  
 2   sit.carga      164 non-null    object 
 3   qtde.unidades  164 non-null    int64  
 4   qtde.teus      164 non-null    int64  
 5   peso.bruto     164 non-null    float64
 6   peso.liquido   164 non-null    float64
dtypes: float64(2), int64(4), object(1)
memory usage: 9.1+ KB


In [69]:
coluna_numerica_tempo = ['entrada.ate.doc.import', 'doc.import.ate.desembaraco',
    'desembaraco.ate.saida', 'entrada.ate.saida',
    'operador', 'transito', 'recinto', 'porto']

for col in coluna_numerica_tempo:
    tempo[col] = coluna_numerica(tempo[col])

tempo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 164 entries, 0 to 163
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   mes                         164 non-null    int64  
 1   ano                         164 non-null    int64  
 2   tipo.documento              164 non-null    int64  
 3   total.ctr                   164 non-null    int64  
 4   entrada.ate.doc.import      100 non-null    float64
 5   doc.import.ate.desembaraco  100 non-null    float64
 6   desembaraco.ate.saida       100 non-null    float64
 7   entrada.ate.saida           100 non-null    float64
 8   operador                    100 non-null    float64
 9   transito                    100 non-null    float64
 10  recinto                     100 non-null    float64
 11  porto                       100 non-null    float64
dtypes: float64(8), int64(4)
memory usage: 15.5 KB


In [70]:
#vendo mes e ano duplicado nos datasets da abtra
duplicado = carga.groupby(['ano', 'mes']).size()
print(duplicado.unique())

print(carga['sit.carga'].unique())
print(tempo['tipo.documento'].unique())

[2]
['D' 'I']
[1 2]


## 4. Verificando buracos no Clima

Aqui agrupamos as informações por ano e contamos quantos valores ausentes (NaN) existem para cada variável de chuva e vento, podendo identificar períodos críticos como anos em que a estação metereológica pode ter ficado fora do ar ou em manutenção, evitando considerar esses periodos como "sem chuva" ou "sem vento".

In [71]:
#buracos em weather-bertioga
missing_chuva = clima.groupby('Ano')['Chuva_Total'].apply(lambda x: x.isna().sum())
missing_vento = clima.groupby('Ano')['Vento_Max_Mes'].apply(lambda x: x.isna().sum())

print("chuva\n", missing_chuva)
print("vento\n", missing_vento)

chuva
 Ano
2019     1
2020    12
2021    12
2022     3
2023     0
2024     0
2025     5
Name: Chuva_Total, dtype: int64
vento
 Ano
2019     1
2020    12
2021    12
2022     3
2023    10
2024     6
2025     9
Name: Vento_Max_Mes, dtype: int64


## 5. Mapeando categorias e verificando disponibilidade

Para facilitar na análise e a visualização, criamos dicionários que traduzem os termos técnicos para descrições claras do negócio. Na carga diferenciamos o que é operação de cais ('D') e do que é recinto ('I'). No tempo, separamos o processo tradicional ('DI') do novo ('DUIMP'). Aproveitamos para verificar a quantidade real de dados disponíveis para cada tipo de documento, contando quantos registros validos exsitem, que vai ajudar no fluxo da modelagem.

In [72]:
mapa_carga = {'D': 'Descarregada (Cais)', 'I': 'Internalizada (Recinto)'}
carga['descricao_carga'] = carga['sit.carga'].map(mapa_carga)

mapa_docs = {1: 'DI (Declaração Importação)', 2: 'DUIMP (Declaração Única)'}
tempo['descricao_doc'] = tempo['tipo.documento'].map(mapa_docs)

print("disponibilidade por tipo de doc")
print(tempo.groupby('descricao_doc')['total.ctr'].apply(lambda x: (x > 0).sum()))

disponibilidade por tipo de doc
descricao_doc
DI (Declaração Importação)    82
DUIMP (Declaração Única)      18
Name: total.ctr, dtype: int64


## 6. Criando e exportando os datasets

Separamos os datasets para focar onde o clima realmente impacta e evitar sujeira nos dados.

Para o dataset de carga, filtramos a situação 'D' (descarregada), que é a operação de cais que provavelmente mais sofre com chuvas e ventos. Fazemos o merge com o clima e criamos a coluna clima_valido para filtrar os anos ruins depois.

Para o dataset de tempo, pegamos só o tipo '1' (DI) para manter a consistência histórica e fugir dos buracos do 'DUIMP'. Fizemos o mesmo processo de merge e validação.

Após isso, salvamos ambos os datasets para serem usados em uma análise exploratória dos dados.


In [73]:
df_foco_operacao = carga[carga['sit.carga'] == 'D'].copy()

df_final_carga = pd.merge(
    df_foco_operacao,
    clima,
    left_on=['ano', 'mes'],
    right_on=['Ano', 'Mes'],
    how='left'
)
df_final_carga['clima_valido'] = df_final_carga['Chuva_Total'].notna()
df_final_carga.to_csv('analise_carga_descarregada_clima.csv', index=False)
print(len(df_final_carga))

82


In [74]:
df_foco_tempo = tempo[tempo['tipo.documento'] == 1].copy()

df_final_tempo = pd.merge(
    df_foco_tempo,
    clima,
    left_on=['ano', 'mes'],
    right_on=['Ano', 'Mes'],
    how='left'
)
df_final_tempo['clima_valido'] = df_final_tempo['Chuva_Total'].notna()
df_final_tempo.to_csv('analise_tempo_DI_clima.csv', index=False)
print(len(df_final_tempo))

82
